# Redemption Analysis

Indigo Protocol is a synthetics based protocol on Cardano. Users can mint synthetic assets like iBTC/iETH/iUSD by providing ADA as collateral.In Indigo v2, the interest rate concept was introduced along with redemptions and v2.1 onwards an algorithmic interest was introduced, which varies according to the state of the protocol iAssets. 

This notebook discussed the steps involved in the generation of data for the algo interest dynamic model(iUSD_algo_model.xlsx) using various internal and external API's. 

This model creates dynamic model showcasing the interest rate trend based on the historical protocol data. 
This notebook focuses specifically on calculating the redeemable amount of iUSD in the protocol as per historical dates. It is one of the inputs of the algo interest mechanism. CDPs with CR between (LR,RMR) are redeemable upto RMR.

This model was an integral part of the alog interest modelling process and was used extensively by the Indigo team to finalize algo interest parameters for all the iAssets.

In [10]:
import requests
import urllib.request
import pandas as pd
import json
from datetime import datetime
from typing import Final
import time
import glob


***Calculating the amount of redeemable iUSD based on various inputs***

In [11]:
#Inputs
interest_rate = 20.075
interest_rate_introduced = 1714320140000
timestamp = 1720117800
ADA_price = 0.363161
iAsset_price = 0.999155
LR = 1.2
RMR = 1.85

# Fetch data from API
headers = {
    'content-type': 'application/json',
}

json_data = {
    'timestamp': timestamp
}

response = requests.post('https://analytics.indigoprotocol.io/api/cdps', headers=headers, json=json_data)
ans = response.json()

df3 = pd.DataFrame(ans)
df3 = df3.drop(['output_hash', 'output_index', 'fee_lovelaces_indy_stakers', 'fee_lovelaces_treasury', 'owner'], axis=1)

# Filter for 'iUSD' asset
df4 = df3[df3['asset'] == 'iUSD'].copy()  # Use `.copy()` to avoid SettingWithCopyWarning

# Modify columns using .loc[]
# Modify columns using .loc[], ensuring the correct dtype
df4.loc[:, 'collateralAmount'] = df4['collateralAmount'].astype(float) / 1000000
df4.loc[:, 'mintedAmount'] = df4['mintedAmount'].astype(float) / 1000000
df4.loc[:, 'interest_iasset_amount'] = df4['interest_iasset_amount'].astype(float) / 1000000

# Calculate interest accrued and corresponding CR, ensuring dtype consistency
df4.loc[:, 'interest_last_updated'] = df4.apply(
    lambda x: interest_rate_introduced if x['interest_last_updated'] < interest_rate_introduced else x['interest_last_updated'],
    axis=1
)

df4.loc[:, 'interest_time'] = (timestamp * 1000) - df4['interest_last_updated']
df4['interest_time'] = df4['interest_time'].astype(float)  # Ensure this column is float type

df4.insert(6, "adjustedinterestAmount", df4['interest_iasset_amount'] + (
    (df4['interest_time'] / (1000 * 60 * 60 * 24 * 365)) * (interest_rate / 100) * df4['mintedAmount']
).astype(float))  # Explicitly cast the calculation result to float

df4.insert(3, "adjustedcollateralAmount", ((df4['collateralAmount'] - (
    df4['adjustedinterestAmount'] * iAsset_price / ADA_price
)).astype(float)))  # Ensure float type for calculated results

df4.insert(7, "CR", (((df4['adjustedcollateralAmount'] * ADA_price) / (
    df4['mintedAmount'] * iAsset_price
)).astype(float)))  # Ensure float type for CR calculation

#calculating redeemable amount of iUSD
df5 = df4[(df4['CR'] > 1.2) & (df4['CR'] < RMR)]
df5.insert(5, column="reedemable", value=(1/(RMR-1))*((RMR*df5['mintedAmount'])-((df5['adjustedcollateralAmount']*ADA_price)/iAsset_price)))
print("Redeemable iUSD: "+str(df5['reedemable'].sum()))

#filtering dataframe
df6 = df5.sort_values(by=['CR'], ascending=True)
columns_to_drop = ['interest_last_updated']
df6 = df6.drop(columns=columns_to_drop)
# print(df6.to_string())
print("Redeemable CDPs: "+ str(df6['CR'].count()))


Redeemable iUSD: 13675.977217450523
Redeemable CDPs: 133


C:\Users\advai\AppData\Local\Temp\ipykernel_7884\2017601759.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2.00000000e+01 1.00000000e+01 1.68707414e+02 5.00000000e+01
 5.00000000e+01 3.30000000e+01 2.51709290e+01 5.00000000e+01
 1.45410960e+01 5.11000000e+02 1.81658494e+02 2.50000000e+03
 3.00000000e+01 1.00000000e+02 1.50000000e+01 1.00000000e+01
 1.00000000e+02 2.61203820e+01 3.00000000e+02 6.29324470e+01
 1.00000000e+01 9.70000000e+02 1.80000000e+01 3.24490320e+01
 9.45482000e+01 4.00000000e+01 4.00000000e+01 5.87650284e+02
 1.00000000e+01 1.26626162e+02 1.00000000e+01 1.23950482e+02
 2.11193110e+01 1.18044430e+04 2.00000000e+01 1.00000000e+01
 5.00000000e+01 2.00000000e+01 6.17855080e+01 5.00000000e+01
 1.48362839e+02 5.00000000e+01 3.50706192e+03 1.00000000e+01
 1.00000000e+01 1.00000000e+02 1.00000000e+01 1.00000000e+01
 1.50000000e+01 1.00000000e+01 1.00000000e+01 1.00000000e+01
 1.00000000e+06 1.5609

**The results obtained above are:**

Redeemable iUSD: 13675.977217450523

Redeemable CDPs: 133

**These are used as inputs in the algo interest model.**